In [15]:
import scipy.stats as stats
import numpy as np
import pandas as pd
from tkinter import filedialog
import matplotlib.pyplot as plt
import seaborn as sns
import os
from math import ceil
from copy import deepcopy
from rich.progress import track
from statannot import add_stat_annotation

In [16]:
# parameters
consecutive_window = 3
threshold_alpha = 1
threshold_D = 0.15 # um2/s
threshold_R2 = 0.5
threshold_step_size = 5 # pixels

In [17]:
def flag_tracks(df):
    track_flags = {}
    step_flags = []
    trackIDs = df["trackID"].unique()

    for track_id in trackIDs:
        track_data = df[df["trackID"] == track_id]
        high_alpha = track_data["alpha"] > threshold_alpha
        step_flags_pertrack = np.repeat(False, track_data.shape[0])

        for i in range(len(high_alpha) - (consecutive_window - 1)):
            if all(high_alpha[i : i + consecutive_window]):
                if (
                    np.mean(track_data["D"][i : i + consecutive_window]) > threshold_D
                    and np.mean(track_data["R2"][i : i + consecutive_window])
                    > threshold_R2
                    and np.mean(track_data["step_sizes"][i : i + consecutive_window])
                    > threshold_step_size
                ):
                    track_flags[track_id] = True
                    step_flags_pertrack[i : i + consecutive_window] = True
        step_flags.extend(step_flags_pertrack)

    df_out = deepcopy(df)
    df_out["step_flag"] = step_flags

    return df_out, track_flags

In [18]:
csv_file_path = filedialog.askopenfilename(
    title="Select CSV File",
    filetypes=(("CSV files", "*.csv"), ("All files", "*.*")),
)

In [19]:
df = pd.read_csv(csv_file_path)
df_out, track_flags = flag_tracks(df)

In [20]:
print(len(track_flags))

1


In [21]:
df_out.to_csv(csv_file_path[:-4] + "-wstepflags.csv", index=False)
print(len(track_flags))
for track_id, flagged in track_flags.items():
    if flagged:
        print(track_id)

1
6
